In [7]:
!git clone https://github.com/kechsin/TinyDialogues.git

fatal: destination path 'TinyDialogues' already exists and is not an empty directory.


In [ ]:
#!mkdir -p ~/miniconda3
#!wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O ~/miniconda3/miniconda.sh
#!bash ~/miniconda3/miniconda.sh -b -u -p ~/miniconda3
#!rm ~/miniconda3/miniconda.sh
#!export PATH="~/miniconda3/condabin:$PATH"
#!source ~/.bashrc
#!conda init

--2025-12-25 19:58:36--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.32.241, 104.16.191.158, 2606:4700::6810:bf9e, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.32.241|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 156772981 (150M) [application/octet-stream]
Saving to: ‘/root/miniconda3/miniconda.sh’

/root/miniconda3/mi 100%[===================>] 149.51M   277MB/s    in 0.5s    

2025-12-25 19:58:37 (277 MB/s) - ‘/root/miniconda3/miniconda.sh’ saved [156772981/156772981]

PREFIX=/root/miniconda3
Unpacking bootstrapper...
Unpacking payload...

Installing base environment...

Preparing transaction: ...working... done
Executing transaction: ...working... done
installation finished.
    You currently have a PYTHONPATH environment variable set. This may cause
    unexpected behavior when running the Python interpreter in Miniconda3.
    For best results, please ve

In [8]:
!pip uninstall -y transformers tokenizers
!pip install transformers==4.57.3 tokenizers==0.22.0

Found existing installation: transformers 4.57.3
Uninstalling transformers-4.57.3:
  Successfully uninstalled transformers-4.57.3
Found existing installation: tokenizers 0.22.0
Uninstalling tokenizers-0.22.0:
  Successfully uninstalled tokenizers-0.22.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.0 MB/s eta 0:00:00
  Using cached tokenizers-0.22.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 148.3 MB/s eta 0:00:00
Using cached tokenizers-0.22.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.3 MB)


In [4]:
#!conda create -n babyLM_train python=3.10.9
#!conda activate babyLM_train
# !pip install TinyDialogues/transformers/
#!cd examples/pytorch/language-modeling
#!pip install -r requirements.txt
#!pip install accelerate tokenizers nltk
#!pip install numpy==1.24.2

Processing ./TinyDialogues/transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached tokenizers-0.13.3.tar.gz (314 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.29.0.dev0-py3-none-any.whl size=6934939 sha256=1e6e8fc8af4c216cdf9d8bb1a6b34948157d732cce88d21b5b4e8e1ec97f78f9
  Stored in directory: /tmp/pip-ephem-wheel-cache-u8sz_khp/wheels/41/09/39/eaf99f581fa8a79939d09d99ab375bc8677109a75d6827d2f1
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tokenizers
Successfully built transformers
Failed to buil

In [5]:
# !pip install tokenizers==0.15.2
# !pip install git+https://github.com/TinyDialogues/transformers.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 51.5 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.1
    Uninstalling tokenizers-0.22.1:
      Successfully uninstalled tokenizers-0.22.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.57.3 requires tokenizers<=0.23.0,>=0.22.0, but you have tokenizers 0.15.2 which is incompatible.
  Cloning https://github.com/TinyDialogues/transformers.git to /tmp/pip-req-build-68fxtn3v
  Running command git clone --filter=blob:none --quiet https://github.com/TinyDialogues/transformers.git /tmp/pip-req-build-68fxtn3v
  fatal: could not read Username for 'https://github.com': No such device or address
  error: subprocess-exited-with-error
  
  × git clone --filter=blob:none --quiet https://github.com/TinyDialogues/transformers.git /tmp/pip-req-build-68fxt

In [6]:
# !pip install -U tokenizers==0.22.0 transformers==4.57.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 59.1 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2


In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.8 MB/s eta 0:00:00


In [ ]:
import os
from datasets import load_dataset

In [ ]:
ds = load_dataset("styfeng/TinyDialogues")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

tinydialogue_train_ordered.txt:   0%|          | 0.00/141M [00:00<?, ?B/s]

tinydialogue_val_ordered.txt:   0%|          | 0.00/25.4M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/110024 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/19708 [00:00<?, ? examples/s]

In [ ]:
ds['train'][0]

{'text': '**Dad**: "Hey sweetie, do you want to paint with Daddy?" \\n\\n **Child**: "Paint!" \\n\\n **Dad**: "Yes, we\'ll use these brushes. But first, let\'s put on your apron so we don\'t get paint on your clothes." \\n\\n **Child**: "Apron!" \\n\\n **Mom**: "Breakfast is almost ready! Who wants pancakes?" \\n\\n **Child**: "Pancake!" \\n\\n **Dad**: "We\'ll eat first, then paint. Let\'s wash hands before we eat, okay?" \\n\\n **Child**: "Wash!" \\n\\n **Mom**: "Careful, the pancakes are hot. We\'ll let them cool a little bit." \\n\\n **Child**: "Hot?" \\n\\n **Mom**: "Yes, hot. But they\'ll be just right soon. Here\'s your plate." \\n\\n **Child**: "Eat pancake!" \\n\\n **Dad**: "After pancakes, we can start our painting. What colors do you want to use?" \\n\\n **Child**: "Blue!" \\n\\n **Mom**: "Blue is a great choice. Finish up and then you can paint with blue." \\n\\n **Child**: "Blue paint!" \\n\\n **Dad**: "Alright, pancakes are done. Let\'s clean up and then it\'s painting ti

In [ ]:
output_folder = "TD"
os.makedirs(output_folder, exist_ok=True)

In [ ]:
with open(os.path.join(output_folder, "train.txt"), "w", encoding="utf-8") as f:
    for text in ds["train"]:
        f.write(text["text"] + "<|endoftext|>" + "\n")

In [ ]:
with open(os.path.join(output_folder, "val.txt"), "w", encoding="utf-8") as f:
    for text in ds["validation"]:
        f.write(text["text"] + "<|endoftext|>" + "\n")

In [ ]:
!python TinyDialogues/scripts/tokenizers/train_GPT2_tokenizer.py TD/train.txt TD/val.txt my_TD_tokenizer

['def', 'Ġadd', '_', 'n', 'umbers', '(', 'a', ',', 'Ġb', '):', 'Ċ', 'Ġ', 'Ġ', 'Ġ', 'Ġ"""', 'Add', 'Ġthe', 'Ġtwo', 'Ġnumbers', 'Ġ`', 'a', '`', 'Ġand', 'Ġ`', 'b', '`', '."', '""', 'Ċ', 'Ġ', 'Ġ', 'Ġ', 'Ġreturn', 'Ġa', 'Ġ+', 'Ġb']
129732
[00:00:00] Tokenize words                 ██████████████████ 93421    /    93421
[00:00:00] Count pairs                    ██████████████████ 93421    /    93421
[00:00:01] Compute merges                 ██████████████████ 51742    /    51742
All special tokens: ['<|endoftext|>', '<UNK>']
BOS token: <|endoftext|>
EOS token: <|endoftext|>
PAD token: None
UNK token: <|endoftext|>
SEP token: None
CLS token: None
MASK token: None


In [ ]:
!python TinyDialogues/scripts/tokenizers/test_GPT2_tokenizer.py my_TD_tokenizer



 TD_output 

['do', 'Ġyou', 'Ġwant', 'Ġto', 'Ġlook', 'Ġat', 'Ġthat', 'Ġit', 'Ġsays', 'Ġlook', 'Ġ', '?']
do you want to look at that it says look ? 

['The', 'Ġyellow', '-', 'billed', 'Ġshri', 'ke', 'Ġ(', "'", 'Cor', 'vin', 'ella', 'Ġcor', 'v', 'ina', "')", 'Ġis', 'Ġa', 'Ġlarge', 'Ġpasser', 'ine', 'Ġbird', 'Ġin', 'Ġthe', 'Ġshri', 'ke', 'Ġfamily', '.', 'ĠIt', 'Ġis', 'Ġsometimes', 'Ġknown', 'Ġas', 'Ġthe', 'Ġlong', '-', 'tailed', 'Ġshri', 'ke', ',', 'Ġbut', 'Ġthis', 'Ġis', 'Ġto', 'Ġbe', 'Ġdiscouraged', ',', 'Ġsince', 'Ġit', 'Ġinvites', 'Ġconfusion', 'Ġwith', 'Ġthe', 'Ġlong', '-', 'tailed', 'Ġshri', 'ke', ',', "Ġ'", 'L', 'an', 'ius', 'Ġsc', 'ha', 'ch', "',", 'Ġof', 'Ġtropical', 'Ġsouthern', 'ĠAsia', '.', 'ĠThe', 'Ġyellow', '-', 'billed', 'Ġshri', 'ke', 'Ġis', 'Ġa', 'Ġcommon', 'Ġresident', 'Ġbreeding', 'Ġbird', 'Ġin', 'Ġtropical', 'ĠAfrica', 'Ġfrom', 'ĠSen', 'egal', 'Ġeast', 'ward', 's', 'Ġto', 'ĠU', 'g', 'anda', 'Ġand', 'Ġlocally', 'Ġin', 'Ġwesternmost', 'ĠK', 'eny', 'a', '.', 'ĠIt', 'Ġfr

Я посмотрела в коде этого теста на то, какие там предложения, и он их правильно написал, всё хорошо.
Можем ещё сравнить с токенайзером обученном ими (на тех же данных, и я ничего не меняла, так что должно быть то же самое)

In [ ]:
!python TinyDialogues/scripts/tokenizers/test_GPT2_tokenizer.py TinyDialogues/tokenizers/GPT2_tinydialogue



 TinyDialogues/tokenizers/GPT2_tinydialogue 

['do', 'Ġyou', 'Ġwant', 'Ġto', 'Ġlook', 'Ġat', 'Ġthat', 'Ġit', 'Ġsays', 'Ġlook', 'Ġ', '?']
do you want to look at that it says look ? 

['The', 'Ġyellow', '-', 'billed', 'Ġshri', 'ke', 'Ġ(', "'", 'Cor', 'vin', 'ella', 'Ġcor', 'v', 'ina', "')", 'Ġis', 'Ġa', 'Ġlarge', 'Ġpasser', 'ine', 'Ġbird', 'Ġin', 'Ġthe', 'Ġshri', 'ke', 'Ġfamily', '.', 'ĠIt', 'Ġis', 'Ġsometimes', 'Ġknown', 'Ġas', 'Ġthe', 'Ġlong', '-', 'tailed', 'Ġshri', 'ke', ',', 'Ġbut', 'Ġthis', 'Ġis', 'Ġto', 'Ġbe', 'Ġdiscouraged', ',', 'Ġsince', 'Ġit', 'Ġinvites', 'Ġconfusion', 'Ġwith', 'Ġthe', 'Ġlong', '-', 'tailed', 'Ġshri', 'ke', ',', "Ġ'", 'L', 'an', 'ius', 'Ġsc', 'ha', 'ch', "',", 'Ġof', 'Ġtropical', 'Ġsouthern', 'ĠAsia', '.', 'ĠThe', 'Ġyellow', '-', 'billed', 'Ġshri', 'ke', 'Ġis', 'Ġa', 'Ġcommon', 'Ġresident', 'Ġbreeding', 'Ġbird', 'Ġin', 'Ġtropical', 'ĠAfrica', 'Ġfrom', 'ĠSen', 'eg', 'al', 'Ġeast', 'ward', 's', 'Ġto', 'ĠU', 'g', 'anda', 'Ġand', 'Ġlocally', 'Ġin', 'Ġwesternmost

Да, вроде одинаково

In [ ]:
!cd TinyDialogues
#/content/TinyDialogues/scripts/language_model_training/GPT2_CHILDES_4-GPUs_train.sh
!bash TinyDialogues/scripts/language_model_training/GPT2_CHILDES_4-GPUs_train.sh \
  TD/train.txt TD/val.txt TD_output/ \
  TinyDialogues/tokenizers/GPT2-small_config GPT2_TD_output \
  gpt2 1e-04 1 8 42

1e-04
gpt2
1
8
42
2025-12-25 20:44:45.241089: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766695485.260122   12550 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766695485.265902   12550 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766695485.280920   12550 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766695485.280945   12550 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766695485.280949   12550 computation_placer.cc:177] comp